# Ch6: Data Encryption Standard (DES)
The Data Encryption Standard (DES) is a symmetric-key algorithm for the encryption of digital data. Although its short key length of 56 bits makes it too insecure for applications, it has been highly influential in the advancement of cryptography.

It takes in a 64 bit data message and a 64 bit key which is then converted into a 48 bit key and new keys are created from the 64 bit key in each iteration of size 48 bit. There are 16 iterations or 16 individual rounds of the Fiestel Cipher in the DES Encryption Algorithm and each iteration uses a different 48 bit key derived from the original 64 bit key.

Each iteration used both a Mixer and a Swapper, wheras the last iteration; the 16th iteration uses only the Mixer.

We define a few utility functions that will be used throughout for converting the integer data to binary strings of fixed block sizes.

In [39]:
def int_to_bin(number: int, block_size=8) -> str:
    binary = bin(number)[2:]
    return '0' * (block_size - len(binary)) + binary


def char_2_num(letter: str) -> int:
    return ord(letter) - ord('a')


def num_2_char(number: int) -> str:
    return chr(ord('a') + number)


def mod(a, b):
    return a % b


def left_circ_shift(binary: str, shift: int) -> str:
    shift = shift % len(binary)
    return binary[shift:] + binary[0: shift]

## 6.1 P-Box 
We Now define a Permutation Box that is used heavily in the DES alorithm. The P-Boxes are of 3 types:

1. Straight P-Box
1. Expansion P-Box
1. Compression P-Box

It will contain the `permutate()` metod that takes a binary string and will return a transposed output, other _static_ methods define the standard P-Boxes used in the DES Algorithm  

In [2]:
class PBox:
    def __init__(self, key: dict):
        self.key = key
        self.in_degree = len(key)
        self.out_degree = sum(len(value) if isinstance(value, list) else 1 for value in key.values())

    def __repr__(self) -> str:
        return 'PBox' + str(self.key)

    def permutate(self, sequence: list) -> str:
        result = [0] * self.out_degree
        for index, value in enumerate(sequence):
            if (index + 1) in self.key:
                indices = self.key.get(index + 1, [])
                indices = indices if isinstance(indices, list) else [indices]
                for i in indices:
                    result[i - 1] = value
        return ''.join(map(str, result))

    def is_invertible(self) -> bool:
        return self.in_degree == self.out_degree

    def invert(self):
        if self.is_invertible():
            result = {}
            for index, mapping in self.key.items():
                result[mapping] = index
            return PBox(result)

    @staticmethod
    def identity(block_size=64):
        return PBox({index: index for index in range(1, block_size + 1)})

    @staticmethod
    def from_list(permutation: list):
        mapping = {}
        for index, value in enumerate(permutation):
            indices = mapping.get(value, [])
            indices.append(index + 1)
            mapping[value] = indices
        return PBox(mapping)

    @staticmethod
    def des_initial_permutation():
        return PBox.from_list(
            [58, 50, 42, 34, 26, 18, 10, 2,
             60, 52, 44, 36, 28, 20, 12, 4,
             62, 54, 46, 38, 30, 22, 14, 6,
             64, 56, 48, 40, 32, 24, 16, 8,
             57, 49, 41, 33, 25, 17, 9, 1,
             59, 51, 43, 35, 27, 19, 11, 3,
             61, 53, 45, 37, 29, 21, 13, 5,
             63, 55, 47, 39, 31, 23, 15, 7]
        )

    @staticmethod
    def des_final_permutation():
        return PBox.from_list(
            [40, 8, 48, 16, 56, 24, 64, 32,
             39, 7, 47, 15, 55, 23, 63, 31,
             38, 6, 46, 14, 54, 22, 62, 30,
             37, 5, 45, 13, 53, 21, 61, 29,
             36, 4, 44, 12, 52, 20, 60, 28,
             35, 3, 43, 11, 51, 19, 59, 27,
             34, 2, 42, 10, 50, 18, 58, 26,
             33, 1, 41, 9, 49, 17, 57, 25]
        )

    @staticmethod
    def des_single_round_expansion():
        """This is the Permutation made on the right half of the block to convert 32 bit --> 42 bits in DES Mixer"""
        return PBox.from_list(
            [32, 1, 2, 3, 4, 5,
             4, 5, 6, 7, 8, 9,
             8, 9, 10, 11, 12, 13,
             12, 13, 14, 15, 16, 17,
             16, 17, 18, 19, 20, 21,
             20, 21, 22, 23, 24, 25,
             24, 25, 26, 27, 28, 29,
             28, 29, 30, 31, 32, 1]
        )

    @staticmethod
    def des_single_round_final():
        """This is the permutation made after the substitution happens in each round"""
        return PBox.from_list(
            [16, 7, 20, 21, 29, 12, 28, 17,
             1, 15, 23, 26, 5, 18, 31, 10,
             2, 8, 24, 14, 32, 27, 3, 9,
             19, 13, 30, 6, 22, 11, 4, 25]
        )

    @staticmethod
    def des_key_initial_permutation():
        return PBox.from_list(
            [57, 49, 41, 33, 25, 17, 9,
             1, 58, 50, 42, 34, 26, 18,
             10, 2, 59, 51, 43, 35, 27,
             19, 11, 3, 60, 52, 44, 36,
             63, 55, 47, 39, 31, 23, 15,
             7, 62, 54, 46, 38, 30, 22,
             14, 6, 61, 53, 45, 37, 29,
             21, 13, 5, 28, 20, 12, 4]
        )

    @staticmethod
    def des_shifted_key_permutation():
        """PC2 Matrix for compression PBox 56 bit --> 48 bit"""
        return PBox.from_list(
            [14, 17, 11, 24, 1, 5, 3, 28,
             15, 6, 21, 10, 23, 19, 12, 4,
             26, 8, 16, 7, 27, 20, 13, 2,
             41, 52, 31, 37, 47, 55, 30, 40,
             51, 45, 33, 48, 44, 49, 39, 56,
             34, 53, 46, 42, 50, 36, 29, 32]
        )

In [4]:
# We can create an expansion box that will expand 32 Bits --> 48 Bits
expansion_p_box = PBox.des_single_round_expansion()
permutation = expansion_p_box.permutate(int_to_bin(1234, block_size=32))
print('Permutation:', permutation)
print('Output Length:', len(permutation))

Permutation: 000000000000000000000000000000001001011010100100
Output Length: 48


In [6]:
# We can create a straight P-Box
straight_p_box = PBox.from_list([4, 1, 3, 2])
p = straight_p_box.permutate('help')
print(p)

phle


In [7]:
# we can also create compression P-Boxes
compression_box = PBox.from_list([3, 3, 2, 1])
p = compression_box.permutate('love')
print(p)

vvol


## 6.2 S-Box
Substitution Boxes use a table to compare __row__ and __column__ that is derived from a given binary string using a function that will return a tuple (__row__, __column__). The output is also a binary string.

We define in our SBox our substitution table and the function that divides input into __row__ and __column__.

In [8]:
class SBox:
    def __init__(self, table: dict, block_size=4, func=lambda binary: (binary[0] + binary[5], binary[1:5])):
        self.table = table
        self.block_size = block_size
        self.func = func

    def __call__(self, binary: str) -> str:
        a, b = self.func(binary)
        a, b = int(a, base=2), int(b, base=2)
        if (a, b) in self.table:
            return int_to_bin(self.table[(a, b)], block_size=self.block_size)
        else:
            return binary

    @staticmethod
    def des_single_round_substitutions():
        return [SBox.forDESSubstitution(block) for block in range(1, 9)]

    @staticmethod
    def identity():
        return SBox(func=lambda binary: ('0', '0'), table={})

    @staticmethod
    def forDESSubstitution(block):
        if block == 1: return SBox.des_s_box1()
        if block == 2: return SBox.des_s_box2()
        if block == 3: return SBox.des_s_box3()
        if block == 4: return SBox.des_s_box4()
        if block == 5: return SBox.des_s_box5()
        if block == 6: return SBox.des_s_box6()
        if block == 7: return SBox.des_s_box7()
        if block == 8: return SBox.des_s_box8()

    @staticmethod
    def des_confusion(binary: str) -> tuple:
        """"Takes a 6-bit binary string as input and returns a 4-bit binary string as output"""
        return binary[0] + binary[5], binary[1: 5]

    @staticmethod
    def from_list(sequence: list):
        mapping = {}
        for row in range(len(sequence)):
            for column in range(len(sequence[0])):
                mapping[(row, column)] = sequence[row][column]
        return SBox(table=mapping)

    @staticmethod
    def des_s_box1():
        return SBox.from_list(
            [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
             [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
             [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
             [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]]
        )

    @staticmethod
    def des_s_box2():
        return SBox.from_list(
            [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
             [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
             [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
             [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]]
        )

    @staticmethod
    def des_s_box3():
        return SBox.from_list(
            [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
             [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
             [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
             [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]]
        )

    @staticmethod
    def des_s_box4():
        return SBox.from_list(
            [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
             [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
             [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
             [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]]
        )

    @staticmethod
    def des_s_box5():
        return SBox.from_list(
            [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
             [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
             [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
             [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]]
        )

    @staticmethod
    def des_s_box6():
        return SBox.from_list(
            [[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
             [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
             [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
             [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]]
        )

    @staticmethod
    def des_s_box7():
        return SBox.from_list(
            [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
             [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
             [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
             [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]]
        )

    @staticmethod
    def des_s_box8():
        return SBox.from_list(
            [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
             [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
             [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
             [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]]
        )

In [12]:
# We craete a custom SBox with our own function
s_box = SBox(block_size=2, table={
    (0, 0): 5,
    (0, 1): 6,
    (1, 0): 8,
    (1, 1): 7
}, func=lambda x: (x[0], x[1]))

print(s_box('00'))
print(s_box('01'))
print(s_box('10'))
print(s_box('11'))

101
110
1000
111


In [14]:
# We can use inbuilt des 1st substitution SBox to compress 6 bit --> 4 bit binary strings
s_box2 = SBox.des_s_box1()
binary = '000100'
print(s_box2(binary))

1101


## 6.3 Swapper
We now define a swapper which will take in a binary string. Divide the string into 2 parts and returns a swapped binary string.

In [15]:
class Swapper:
    def __init__(self, block_size=64):
        self.block_size = block_size

    def encrypt(self, binary: str) -> str:
        l, r = binary[0: self.block_size // 2], binary[self.block_size // 2:]
        return r + l

    def decrypt(self, binary: str) -> str:
        return self.encrypt(binary)

In [16]:
swapper = Swapper(block_size=8)
ciphertext = swapper.encrypt('11110000')
print(ciphertext)

00001111


In [17]:
# We can also decrypt this encrypted binary string
print(swapper.decrypt(ciphertext))

11110000


In [20]:
# We can create a 64 bit swapper that is used extensively in the DES algorithm
swapper_64_bit = Swapper()
ciphertext = swapper_64_bit.encrypt(int_to_bin(100, block_size=64))
print(ciphertext)

0000000000000000000000000110010000000000000000000000000000000000


In [21]:
# We now decrypt it
int(swapper_64_bit.decrypt(ciphertext), base=2)

100

## 6.4 Null Swapper
The Null Swapper isn't a cryptographical object but rather an Object we have created to have the same API as the `Swapper` so that we can plug in the `NoneSwapper` in place of the `Swapper` class when we wish to create a cipher that doesn't swap bit blocks.

> It serves no purpose in the algorithm. It simple provides us a mechanism to create a Fiestel Round wihout swapping which we shall see later on.

In [22]:
class NoneSwapper:
    def encrypt(self, binary: str) -> str:
        return binary

    def decrypt(self, binary: str) -> str:
        return binary

In [23]:
# It doesn't change the binary block during encryption or decryption
swapper = NoneSwapper()
swapper.encrypt('1001')

'1001'

In [24]:
swapper.decrypt('1001')

'1001'

## 6.5 Mixer
The Mixer takes in a binary block of fixed length and divides in into 2 parts. The right part is then combined with the key and  a non-invertible function is performed on the right part and key. The result is then XORed (^) with the left part.

The end results are
```text
l = l ^ f(r, K)
r = r
```

In [25]:
class Mixer:
    def __init__(self, key: int, func=lambda a, b: a % b, block_size=64,
                 initial_permutation=None, final_permutation=None,
                 substitutions: list = None, substitution_block_size=6):
        self.func = func
        self.block_size = block_size
        self.initial_permutation = PBox.identity(block_size // 2) if initial_permutation is None else initial_permutation
        self.final_permutation = PBox.identity(block_size // 2) if final_permutation is None else final_permutation
        self.substitutions = SBox.des_single_round_substitutions() if substitutions is None else substitutions
        self.substitution_block_size = substitution_block_size
        self.key = key

    def encrypt(self, binary: str) -> str:
        l, r = binary[0: self.block_size // 2], binary[self.block_size // 2:]
        # expansion PBox
        r1: str = self.initial_permutation.permutate(r)

        # applying function
        r2: str = int_to_bin(self.func(int(r1, base=2), self.key), block_size=self.initial_permutation.out_degree)

        # applying the substitution matrices
        r3: str = ''
        for i in range(len(self.substitutions)):
            block: str = r2[i * self.substitution_block_size: (i + 1) * self.substitution_block_size]
            r3 += self.substitutions[i](block)

        # applying final permutation
        r3: str = self.final_permutation.permutate(r3)

        # applying xor
        l = int_to_bin(int(l, base=2) ^ int(r3, base=2), block_size=self.block_size // 2)
        return l + r

    def decrypt(self, binary:str) -> str:
        return self.encrypt(binary)

    @staticmethod
    def des_mixer(key: int):
        return Mixer(
          key=key,
          initial_permutation=PBox.des_single_round_expansion(),
          final_permutation=PBox.des_single_round_final(),
          func=lambda a, b: a % b
        )

In [26]:
# We craete a DES specific mixer. That means that the block_size will be 64 and DES specific PBoxes and SBoxes will be used.
# Also we use the mod function when performing a non-invertible operation over r ad Key hence f = r % Key
mixer = Mixer.des_mixer(key=3)
number = 1234
binary = int_to_bin(number, block_size=64)
print('Plaintext:', binary)

Plaintext: 0000000000000000000000000000000000000000000000000000010011010010


In [28]:
ciphertext = mixer.encrypt(binary)
print('Ciphrtext:', ciphertext)

Ciphrtext: 1101000011011010110100111001110000000000000000000000010011010010


In [31]:
# decrypting using the Mixer
decrypted = mixer.decrypt(ciphertext)
print('Decrypted:', decrypted)

Decrypted: 0000000000000000000000000000000000000000000000000000010011010010


In [32]:
# printing the integer based output
print(int(decrypted, base=2))

1234


## 6.6 Round
A Sigle Fiestel Round employs both the Mixer and the Swapper Cipher. Both ciphers are performed in succession (one after the other) __Mixer__ --> __Swapper__ and the inverse during decryption.

We can also have a Fiestel Round with just the Mixer and no swapping, which will be equivalent to just one Mixer. We define 2 factory methods:

1. `with_swapper`: That returns a Fiestel Round Cipher object that uses both a Mixer and a Swapper to encrypt and decrypt data.
1. `without_swapper`: That returns a Fiestel Round Cipher object that usesonly  a Mixer to encrypt and decrypt data.

In [33]:
class Round:
    def __init__(self, mixer):
        self.mixer = mixer
        self.swapper = NoneSwapper()

    @staticmethod
    def with_swapper(mixer: Mixer):
        temp = Round(mixer)
        temp.swapper = Swapper(block_size=mixer.block_size)
        return temp

    @staticmethod
    def without_swapper(mixer: Mixer):
        return Round(mixer)

    def encrypt(self, binary: str) -> str:
        binary = self.mixer.encrypt(binary)
        return self.swapper.encrypt(binary)

    def decrypt(self, binary: str) -> str:
        binary = self.swapper.decrypt(binary)
        return self.mixer.decrypt(binary)

In [34]:
number = 12345
binary = int_to_bin(number, block_size=64)
round1 = Round.without_swapper(Mixer.des_mixer(key=17))
ciphertext = round1.encrypt(binary)
print('Ciphertext:', ciphertext)

Ciphertext: 1101000011011000110100111011110000000000000000000011000000111001


In [35]:
print('Decrypted Number:', int(round1.decrypt(ciphertext), base=2))

Decrypted Number: 12345


In [36]:
# We can also create a Fiestel Round with swapping
number = 12345
binary = int_to_bin(number, block_size=64)
round1 = Round.with_swapper(Mixer.des_mixer(key=17))
ciphertext = round1.encrypt(binary)
print('Ciphertext:', ciphertext)

Ciphertext: 0000000000000000001100000011100111010000110110001101001110111100


In [37]:
print('Decrypted Number:', int(round1.decrypt(ciphertext), base=2))

Decrypted Number: 12345


## 6.6 DES
We now impliment the DES algorithm which consists of 16 Fiestel rounds. We use the predefined `SBox` and `PBox` classes and also the `Round` cipher class. 

In [50]:
class DES:
    def __init__(self, key: int):
        self.key = int_to_bin(key, block_size=64)
        self.PC_1 = PBox.des_key_initial_permutation()
        self.PC_2 = PBox.des_shifted_key_permutation()
        self.single_shift = {1, 2, 9, 16}
        self.rounds = self.generate_rounds()

    def encrypt(self, binary: str) -> str:
        for round in self.rounds:
            binary = round.encrypt(binary)
        return binary

    def decrypt(self, binary: str) -> str:
        for round in self.rounds[::-1]:
            binary = round.decrypt(binary)
        return binary

    def encrypt_message(self, plaintext: str) -> list:
        result = [0] * len(plaintext)
        for index, letter in enumerate(plaintext.lower()):
            result[index] = int(self.encrypt(int_to_bin(ord(letter), block_size=64)), base=2)
        return result

    def decrypt_message(self, ciphertext_stream: list) -> str:
        return ''.join(map(chr, self.plaintext_stream(ciphertext_stream)))

    def plaintext_stream(self, ciphertext_stream: list) -> list:
        return [int(self.decrypt(int_to_bin(number, block_size=64)), base=2) for number in ciphertext_stream]

    def generate_rounds(self) -> list:
        rounds = []
        self.key = self.PC_1.permutate(self.key)
        l, r = self.key[0: 32], self.key[32:]
        for i in range(1, 17):
            shift = 1 if i in self.single_shift else 2
            l, r = left_circ_shift(l, shift), left_circ_shift(r, shift)
            key = int(self.PC_2.permutate(l + r), base=2)
            mixer = Mixer.des_mixer(key)
            cipher = Round.with_swapper(mixer) if i != 16 else Round.without_swapper(mixer)
            rounds.append(cipher)
        return rounds

In [51]:
# We create an instance of DES with a key and use it to enciphr and decipher 64 bit binary data
number = 12345
binary = int_to_bin(number, block_size=64)
des = DES(key=78)
ciphertext = des.encrypt(binary)
print('Plaintext:', binary)
print('Ciphertext:', ciphertext)

Plaintext: 0000000000000000000000000000000000000000000000000011000000111001
Ciphertext: 0111100100111011001111010010010100101001101110001101101010100110


In [52]:
# We now decipher the ciphertext
decrypted = des.decrypt(ciphertext)
print('Decrypted:', decrypted)

Decrypted: 0000000000000000000000000000000000000000000000000011000000111001


In [53]:
# Checking value of decrypted
print('Value:', int(decrypted, base=2))

Value: 12345


In [55]:
# We can also use the encrypt_message API to encrypt and decrypt string messages
message = 'hello world 😀'
ciphertext = des.encrypt_message(message)
print('Plaintext:', message)
print('Ciphertext:', ciphertext)

Plaintext: hello world 😀
Ciphertext: [9253891819891440400, 14118313512560970, 3982981733458429922, 3982981733458429922, 16835800272982933939, 16367344448200892497, 14303274998317469877, 16835800272982933939, 10048292260086069323, 3982981733458429922, 5262255425266249560, 16367344448200892497, 4133802428460097780]


In [56]:
# We see that teh ciphertext we get above is a stream of 64 bit integer numbers. We can also decrypt our message
decrypted = des.decrypt_message(ciphertext)
print('Decrypted:', decrypted)

Decrypted: hello world 😀
